## 초기 환경 세팅 및 확인
1. requirements.txt 및 MLflow 설치
2. torch는 cuda버전을 사용하고 있으므로 사용자가 추가로 다운로드 필요
3. 그래픽카드 사용 가능여부 확인
    - get_device_name에서 안뜨면 그래픽카드 안잡힌것.
    - 만약 안잡히면 !nvidia-smi 주석을 풀고 그래픽카드가 제대로 잡히는지 확인

In [ ]:
%pip install --upgrade pip
%pip install -r requirements.txt

### 그래픽 카드 확인
**torch로 그래픽카드가 잡히지 않을 때만 확인용으로 주석 제거 후 돌리기**

토치 버전 확인

In [ ]:
%pip show torch

CUDA 확인

In [1]:
import torch

print(torch.cuda.get_device_name(0))

#!nvidia-smi

NVIDIA GeForce RTX 4050 Laptop GPU


## 모델 가져오기
- 현재 파일에서는 hugginface의 beomi/KcELECTRA-base 모델을 사용할 예정입니다.
- 다른 모델을 사용하기 원할 경우 huggingface의 가이드에 따라 모델명을 변경하면 됩니다.

In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForPreTraining


### 성능 향상을 위한 Text 정제
- huggingface에서 개발자가 공개한 성능을 높이기 위한 데이터 전처리 과정입니다. 특수문자 및 이모지 등을 제거합니다

In [2]:
import re
import emoji
from soynlp.normalizer import repeat_normalize

emojis = ''.join(emoji.EMOJI_DATA.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

import re
import emoji
from soynlp.normalizer import repeat_normalize

pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

def clean(x): 
    x = pattern.sub(' ', x)
    x = emoji.replace_emoji(x, replace='') #emoji 삭제
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x

# KcELECTRA 확인 코드

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification


model_name = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained("beomi/KcELECTRA-base-v2022-klue-nsmc")

sentence = "오늘 하루는 정말 힘들었지만 그래도 조금은 나아진 것 같아."
cleaned_sentence = clean(sentence)
print('clean 함수가 적용된 문장 : ',cleaned_sentence)


# 추론
inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.softmax(logits, dim=1)
    pred = torch.argmax(probs, dim=1).item()

# 결과 출력
labels = ["부정", "긍정"]
print(f"문장: {sentence}")
print(f"예측 감정: {labels[pred]} (확률: {probs[0][pred]:.4f})")




tokenizer_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

c:\Users\SSAFY\Desktop\WANG\Kcvenv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SSAFY\.cache\huggingface\hub\models--beomi--KcELECTRA-base-v2022. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

OSError: beomi/KcELECTRA-base-v2022-klue-nsmc is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

# KoBERT-Senti5 모델 확인 코드

In [ ]:
from transformers import AutoTokenizer, BertForSequenceClassification

# KoBERT의 원래 토크나이저 사용
tokenizer = AutoTokenizer.from_pretrained('monologg/kobert')
model = BertForSequenceClassification.from_pretrained('jeonghyeon97/koBERT-Senti5')

# 예시 입력 (여러 문장 리스트)
texts = [
    "오늘은 정말 행복한 하루야!",
    "이거 정말 짜증나고 화난다.",
    "그냥 그렇네.",
    "왜 이렇게 슬프지?",
    "기분이 좀 불안해."
]

# 입력 텍스트 토큰화
inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)

# 예측
outputs = model(**inputs)
predictions = outputs.logits.argmax(dim=-1)

# 결과 출력
for text, prediction in zip(texts, predictions):
    print(f"입력: {text} -> 예측된 감정 레이블: {prediction.item()}")

## 학습 시킬 데이터 셋. json 형식으로 준비할 것.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="data/train.json")

def format_example(example):
    return tokenizer(
        f"<|user|>: {example['instruction']}\n<|assistant|>: {example['output']}",
        truncation=True,
        padding="max_length",
        max_length=512,
    )

tokenized_dataset = dataset.map(format_example)


## 학습 실행

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./qwen3-finetuned",     # 여기로 파인튜닝된 모델 저장
    per_device_train_batch_size=2,      
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=3,
    fp16=True,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)
trainer.train()


## LORA 가중치 병합

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # Qwen은 Q/V projection에 적용
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# 파라미터가 1~200만 정도 수준이면 잘 된거
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
model.save_pretrained("./qwen3-lora")
tokenizer.save_pretrained("./qwen3-lora")

from peft import merge_and_unload
merged_model = merge_and_unload(model)
merged_model.save_pretrained("./qwen3-merged")
